# Evaluating the GPT models on the SPOT Dataset
# Currently using the Chat.OpenAI.GPT3.5/4  Engine

## This requires manually copying questions into the browser and waiting for response
## This has also been implemented using the openAI.API, however this charges on a per token basis. OpenAI provide a free starting $20 budget. I used it prior to the significant price decrease and exhausted my budget.


In [1]:
# Use the experiment output from one of the EleutherAI GPT-Neo models to generate the list of questions to pass into the Chat.OpenAI interface
# We use the version of Eleuther that processes Yes/No questions with zero-shot context
import pandas as pd
import ujson as json
import pprint
import os
import yaml

pp = pprint.PrettyPrinter(indent=4, width=260)
experiment_output = pd.read_csv('eleuther_predictions.csv')

In [2]:
import openai
openai.api_key = open('./openai.key.temp','r').read()

from util_openai import OpenAICompletion

In [3]:
# Preprocessing Questions to Pass to Chat.OpenAI
preds_prompts = experiment_output['preds_prompts'].tolist()
preds_prompts = [json.decode(x) for x in preds_prompts]

# Remove the text 'Question:' from the start of each prompt
preds_prompts = [ x[0].replace('Question: ', '') for x in preds_prompts]

# Remove double qoutation marks in each prompt
preds_prompts = [ x.replace('""', '"') for x in preds_prompts]

# Remove the text 'Answer:' from the start of each prompt
preds_prompts_direct = [x.replace('\nAnswer:', '') for x in preds_prompts]
preds_prompts_indirect = [x.replace('directly', 'indirectly') for x in preds_prompts_direct]
preds_prompts_normal = [x.replace('directly ', '') for x in preds_prompts_direct]


In [4]:
oac = OpenAICompletion(
    openai_model='gpt-3.5-turbo',
    max_tokens=5,
    temperature=1.0,
    system_start='Assistant is an intelligent chatbot designed to help users answers questions about whether government spending on "budget item" affects a "socio-economic indicator".')

In [5]:
responses_direct = oac.get_responses(preds_prompts_direct)
responses_indirect = oac.get_responses(preds_prompts_indirect)
responses_normal = oac.get_responses(preds_prompts_normal)

 57%|█████▋    | 410/725 [19:50<14:29,  2.76s/it]  

Rate limit reached for default-gpt-3.5-turbo in organization org-fPO4EPPHMh44ZiLPLQMVeppD on requests per min. Limit: 20 / min. Please try again in 3s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.


 34%|███▍      | 246/725 [12:01<22:01,  2.76s/it]  

Rate limit reached for default-gpt-3.5-turbo in organization org-fPO4EPPHMh44ZiLPLQMVeppD on requests per min. Limit: 20 / min. Please try again in 3s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.


 99%|█████████▉| 716/725 [35:46<00:24,  2.77s/it]  

Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 725/725 [35:35<00:00,  2.95s/it]  


In [6]:
# Converting Responses to Yes/No/Nan
def convert_to_yes_no_nan(response):
    if 'no' in response.lower():
        return 'No'
    if 'yes' in response.lower():    
        return 'Yes'
    else:
        return 'NA'

responses_direct = [convert_to_yes_no_nan(x) for x in responses_direct]
responses_indirect = [convert_to_yes_no_nan(x) for x in responses_indirect]
responses_normal = [convert_to_yes_no_nan(x) for x in responses_normal]

In [7]:
import copy
# Save Predictions to File
# Drop the following columns from experiment_output, preds_ensemble_parsed, preds_ensemble, preds_prompts
experiment_output = experiment_output.drop(columns=['preds_ensemble_parsed', 'preds_ensemble', 'preds_prompts','preds_aggregate'], axis=1, errors='ignore')

# Setting up the experiment_output dataframe to save the predictions
experiment_output_direct = copy.deepcopy(experiment_output)
experiment_output_indirect = copy.deepcopy(experiment_output)
experiment_output_normal = copy.deepcopy(experiment_output)

# Add the predictions to the experiment_output dataframes
experiment_output_direct['preds_aggregate'] = responses_direct
experiment_output_direct['preds_prompts'] = [json.encode(x) for x in preds_prompts_direct]

experiment_output_indirect['preds_aggregate'] = responses_indirect
experiment_output_indirect['preds_prompts'] = [json.encode(x) for x in preds_prompts_indirect]

experiment_output_normal['preds_aggregate'] = responses_normal
experiment_output_normal['preds_prompts'] = [json.encode(x) for x in preds_prompts_normal]

# Create the config files
config_direct = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'directly'}
config_indirect = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'indirectly'}
config_normal = {'dset_name':'spot', 'k_shot':0, 'nn_name':'gpt-3.5-turbo', 'directly_or_indirectly':'none'}

In [8]:
#Creating file directories
dir_ = '../output'

dir_direct = 'spot_gpt35turbo_directly'
dir_indirect = 'spot_gpt35turbo_indirectly'
dir_normal = 'spot_gpt35turbo'

os.makedirs(os.path.join(dir_, dir_direct), exist_ok=True)
os.makedirs(os.path.join(dir_, dir_indirect), exist_ok=True)
os.makedirs(os.path.join(dir_, dir_normal), exist_ok=True)

# Saving the experiment_output dataframes 
experiment_output_direct.to_csv(os.path.join(dir_, dir_direct, 'predictions.csv'),index=False)
experiment_output_indirect.to_csv(os.path.join(dir_, dir_indirect, 'predictions.csv'),index=False)
experiment_output_normal.to_csv(os.path.join(dir_, dir_normal, 'predictions.csv'),index=False)

# Saving the config files
with open(os.path.join(dir_, dir_direct, 'config.yaml'), 'w') as f:
    yaml.dump(config_direct, f)
    
with open(os.path.join(dir_, dir_indirect, 'config.yaml'), 'w') as f:
    yaml.dump(config_indirect, f)
    
with open(os.path.join(dir_, dir_normal, 'config.yaml'), 'w') as f:
    yaml.dump(config_normal, f)
